In [33]:
import base64
from io import BytesIO

import cv2
import urllib
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [34]:
# Path to the Streamlit public S3 bucket
DATA_URL_ROOT = "https://streamlit-self-driving.s3-us-west-2.amazonaws.com/"

# Functions

In [35]:
def get_selected_frames(summary, label, min_elts, max_elts):
    return summary[np.logical_and(summary[label] >= min_elts, summary[label] <= max_elts)].index

In [36]:
def load_image(url):
    with urllib.request.urlopen(url) as response:
        image = np.asarray(bytearray(response.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = image[:, :, [2, 1, 0]] # BGR -> RGB
    return image

In [37]:
def load_network(config_path, weights_path):
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    output_layer_names = net.getLayerNames()
    output_layer_names = [output_layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layer_names

In [38]:
def create_summary(metadata):
    one_hot_encoded = pd.get_dummies(metadata[["frame", "label"]], columns=["label"])
    summary = one_hot_encoded.groupby(["frame"]).sum().rename(columns={
        "label_biker": "biker",
        "label_car": "car",
        "label_pedestrian": "pedestrian",
        "label_trafficLight": "traffic light",
        "label_truck": "truck"
    })
    return summary

In [154]:

def yolo_v3(image, net, output_layer_names, confidence_threshold, overlap_threshold):
    # Run the YOLO neural net.
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(output_layer_names)

    # Supress detections in case of too low confidence or too much overlap.
    boxes, confidences, class_IDs = [], [], []
    H, W = image.shape[:2]
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confidence_threshold:
                box = detection[0:4] * np.array([W, H, W, H])
                centerX, centerY, width, height = box.astype("int")
                x, y = int(centerX - (width / 2)), int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_IDs.append(classID)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, overlap_threshold)

    # Map from YOLO labels to Udacity labels.
    UDACITY_LABELS = {
        0: 'pedestrian',
        1: 'biker',
        2: 'car',
        3: 'biker',
        5: 'truck',
        7: 'truck',
        9: 'trafficLight'
    }
    xmin, xmax, ymin, ymax, labels, scores = [], [], [], [], [], []
    if len(indices) > 0:
        # loop over the indexes we are keeping
        for i in indices.flatten():
            label = UDACITY_LABELS.get(class_IDs[i], None)
            if label is None:
                continue

            # extract the bounding box coordinates
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]

            xmin.append(x)
            ymin.append(y)
            xmax.append(x+w)
            ymax.append(y+h)
            labels.append(label)
            scores.append(confidences[i])

    boxes = pd.DataFrame({
        "xmin": xmin, "ymin": ymin, 
        "xmax": xmax, "ymax": ymax, 
        "label": labels, 'confidence': scores})
    
    return boxes

In [132]:
yolo_boxes = yolo_v3(image, net, output_layer_names, confidence_threshold, overlap_threshold)

# Plot functions

In [40]:
def array_to_b64(img, enc='jpg'):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    is_success, buffer = cv2.imencode(f".{enc}", img)
    io_buf = BytesIO(buffer)
    encoded = base64.b64encode(io_buf.getvalue()).decode("utf-8")
    return f'data:img/{enc};base64, ' + encoded

In [91]:
def add_bbox(fig, x0, y0, x1, y1, color='red', opacity=0.5, name=""):
    fig.add_scatter(
        x=[x0,x1,x1,x0,x0], 
        y=[y0,y0,y1,y1,y0], 
        mode="lines", 
        fill="toself", 
        opacity=opacity,
        marker_color=color,
        hoveron="fills", 
        hoverlabel_namelength=-1,
        name=name
    )

In [151]:
def display_images_with_bbox(image, boxes, title):
    LABEL_COLORS = {
        "car": ['LightSkyBlue', 'RoyalBlue'],
        "pedestrian": ['red', 'darkred'],
        "truck": ['green', 'darkgreen'],
        "trafficLight": ['lightyellow', 'yellow'],
        "biker": ['orange', 'darkorange'],
    }
    img_height, img_width = image.shape[:2]
    
    fig = go.Figure()
    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    fig.add_trace(
        go.Scatter(
            x=[img_width*.05, img_width*.95],
            y=[img_height*.95, img_height*.05],
            mode="markers",
            marker_opacity=0,
            hoverinfo='none'
        )
    )
    fig.add_layout_image(dict(
        source=array_to_b64(image),
        x=0, y=0,
        xref="x", yref="y",
        sizex=img_width, 
        sizey=img_height,
        sizing='stretch',
        opacity=1,
        layer="below"
    ))

    for index, box in boxes.iterrows():
        fill_col, line_col = LABEL_COLORS[box.label]
        add_bbox(
            fig,
            x0=box.xmin, y0=box.ymin,
            x1=box.xmax, y1=box.ymax,
            opacity=0.5,
            color=fill_col,
            name=f'class={box.label}<br>confidence={box.confidence:.3f}'
        )
        
    # Adapt axes to the right width and height, lock aspect ratio
    fig.update_xaxes(showgrid=False, visible=False, constrain='domain', range=[0, img_width])
    fig.update_yaxes(showgrid=False, visible=False, scaleanchor='x', scaleratio=1, range=[img_height, 0])
    fig.update_layout(title=title, showlegend=False)

    return fig

In [90]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_scatter(
    x=[3,4,4,3,3], 
    y=[1,1,2,2,1], 
    mode="lines", fill="toself",
    hoveron="points+fills", hoverinfo="x+y"
)
fig.add_scatter(
    x=[1,2,2,1,1], 
    y=[1,1,2,2,1], 
    mode="lines", fill="toself", opacity=0.5,
    marker_color="green",
    hoveron="fills",
    name="CLASS<BR>confidence",
    hoverlabel_namelength=-1
)
fig.update_xaxes(range=[0,6])
fig.update_yaxes(range=[0,6])
fig.update_layout(showlegend=False)

# Start here 

In [42]:
object_type = 'pedestrian'
confidence_threshold = .5
overlap_threshold = .5
selected_frame_index = 0
min_elts, max_elts = 10, 20

In [137]:
metadata = pd.read_csv("labels.csv.gz")
metadata['confidence'] = 1.0
summary = create_summary(metadata)

In [44]:
selected_frames = get_selected_frames(summary, object_type, min_elts, max_elts)
selected_frame = selected_frames[selected_frame_index]

In [45]:
img_bucket = "https://images.plot.ly/udacity-self-driving-cars/"
image_url = os.path.join(img_bucket, selected_frame)
image = load_image(image_url)

In [152]:
real_boxes = metadata[metadata.frame == selected_frame].drop(columns=["frame"])
real_fig = display_images_with_bbox(image, real_boxes, 'Human Annotated')
real_fig.show()

In [139]:
net, output_layer_names = load_network("yolov3.cfg", "yolov3.weights")

In [153]:
yolo_boxes = yolo_v3(image, net, output_layer_names, confidence_threshold, overlap_threshold)

yolo_fig = display_images_with_bbox(image, yolo_boxes, 'Yolo v3 predictions')
yolo_fig.show()